# Семинар 3. Исправление опечаток

In [20]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)

from sklearn.metrics import classification_report, accuracy_score

Возьмем данные с соревнования Dialog Evaluation 2015 по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

Я удалили из данных случаи, когда в словах пропущен или вставлен пробел, чтобы было проще сопоставить слова в предложении. 

In [144]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [212]:
# Посмотрим на пары предложений
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [146]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [147]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [148]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [149]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.13016983016983016


Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [8]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Новостные тексты для этого хорошо подходят.

Я взял 5000 текстов из корпуса текстов Ленты.ру (скачать целиком можно тут - https://github.com/yutkin/Lenta.Ru-News-Dataset)

Вы можете скачать весь датасет и взять больше текстов. Закоменченные ячейки помогут вам это сделать.

In [9]:
# import gzip
# import csv

In [10]:
# corpus = open('corpus_5000.txt', 'w')
# with gzip.open('lenta-ru-news.csv.gz', 'rt') as archive:
#     reader = csv.reader(archive, delimiter=',', quotechar='"')
#     for i, line in enumerate(reader):
#         if i < 5000: # увеличьте количество текстов тут
#             corpus.write(line[2].replace('\xa0', ' ') + '\n')

In [213]:
def normalize(text):
    
    normalized_text = [(word.strip(punctuation)) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [151]:
corpus = []
for text in open('corpus_5000.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

Попробуем предсказать ошибку простым заглядыванием в словарь. Если слово не в словаре - оно неправильное.

In [152]:
# создаем словарь
vocab = set()

for sent in corpus:
    vocab.update(sent)


In [153]:
def predict_mistaken(word, vocab):

    if word in vocab:
        return 0
    else:
        return 1

    
    

In [105]:
# для оценки создаем два списка y_true и y_pred
# проходимся по предложениям
# сопоставляем слова с помощью функции align_words
# проходимся по парам слов и
# если слова одинаковые добавляем в y_true 0 
# если слова разные добавляем в y_true 1
# предказываем ошибочность слова из bad списка 
# добавляем предсказание в список y_pred

y_true = []
y_pred = []

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] == pair[1]:
            y_true.append(0)
        else:
            y_true.append(1)
        
        y_pred.append(predict_mistaken(pair[1], vocab))
    

Оценим качество.

In [106]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred, ))

             precision    recall  f1-score   support

          0       0.98      0.90      0.94      8707
          1       0.57      0.88      0.69      1303

avg / total       0.93      0.90      0.91     10010



### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известного алгоритма Питера Норвига.

Ошибки вычисляются также, по словарю. Но ещё мы собираем частотности, чтобы потом выбирать самое вероятное исправление.

In [22]:
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [23]:
WORDS.most_common(10)

[('в', 127500),
 ('и', 62292),
 ('на', 53045),
 ('что', 34710),
 ('с', 28887),
 ('по', 26590),
 ('не', 22895),
 ('из', 13579),
 ('он', 13212),
 ('его', 11950)]

In [155]:
"солнце" in WORDS

True

In [156]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N


In [157]:
P('солнце')

1.0886940251721075e-05

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [27]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [158]:
%%time
correction('сонце')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 465 µs


'конце'

In [159]:
word = 'сонце'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

In [160]:
splits[:10]

[('', 'сонце'),
 ('с', 'онце'),
 ('со', 'нце'),
 ('сон', 'це'),
 ('сонц', 'е'),
 ('сонце', '')]

In [161]:
deletes = [L + R[1:] for L, R in splits if R]

In [162]:
deletes[:10]

['онце', 'снце', 'соце', 'соне', 'сонц']

In [163]:
transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

In [164]:
transposes[:10]

['оснце', 'сноце', 'соцне', 'сонец']

In [165]:
letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

In [166]:
len(replaces)

165

In [167]:
inserts = [L + c + R for L, R in splits for c in letters]

In [168]:
inserts[:10]

['йсонце',
 'цсонце',
 'усонце',
 'ксонце',
 'есонце',
 'нсонце',
 'гсонце',
 'шсонце',
 'щсонце',
 'зсонце']

In [39]:
len(inserts)

198

Для оценки используем будем использовать три метрики:  
1) процент правильных слов;  
2) процент исправленных ошибок  
3) процент ошибочно исправленных правильных слов

In [169]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


Получается, что ошибок стало ещё больше.

In [170]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7332667332667333
0.5226400613967767
0.23521304697369932


Ещё проблема тут в том, что алгоритм медленно работает для длинных слов.

In [214]:
%%time
correction('солнце')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 52 µs


'солнце'

In [173]:
%%time
correction('насмехатьсяаававттававаываываы')

CPU times: user 2.16 s, sys: 36 ms, total: 2.19 s
Wall time: 2.19 s


'насмехатьсяаававттававаываываы'

Посмотрим, как исправляются самые частотные ошибки.

In [175]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'себя'),
 ('вообще', 'вобще', 'вообще'),
 ('вообще', 'ваще', 'чаще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'кстати'),
 ('очень', 'ооочень', 'ооочень'),
 ('как-то', 'както', 'какао'),
 ('очень', 'оооочень', 'ооочень'),
 ('это', 'ето', 'что')]

In [216]:
WORDS['ооочень']

1

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования (количество редактирований, которые нужно сделать в строке a, чтобы прийти к b.

Самое известное расстояние редактирования - расстояние Левенштейна. Тут мы не будет разбирать алгоритм, можете почитать [тут](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0), а код на питоне есть [тут](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%BE%D0%B2/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0)

Ещё есть расстояние Дамерау-Левенштейна - почти то же самое, только разрешена ещё операция перестановки.

Про самого Левенштейна можно почитать вот тут - https://nplus1.ru/material/2017/09/25/vladimir-levenshtein

In [45]:
# в питоне есть библиотеке для нахождения близких строк

In [46]:
from difflib import get_close_matches

In [47]:
%%time
get_close_matches('сонце', WORDS.keys(), n=1)

CPU times: user 564 ms, sys: 0 ns, total: 564 ms
Wall time: 560 ms


['солнце']

Работает тоже не очень быстро.

Есть библиотека textdistance, в которой реализованы многие методы нахождения расстояний.

In [48]:
import textdistance

In [49]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

In [176]:
%%time
get_closest_match_with_metric('сонце', WORDS, 3, textdistance.hamming)

CPU times: user 3.05 s, sys: 0 ns, total: 3.05 s
Wall time: 3.05 s


[('конце', 0.8), ('соннен', 0.6666666666666667), ('конца', 0.6)]

In [177]:
%%time
get_closest_match_with_metric('сонце', WORDS, 5, textdistance.damerau_levenshtein)

CPU times: user 30.8 s, sys: 408 ms, total: 31.2 s
Wall time: 29.8 s


[('солнце', 0.8333333333333334),
 ('конце', 0.8),
 ('соне', 0.8),
 ('солнцем', 0.7142857142857143),
 ('солнца', 0.6666666666666667)]

In [181]:
%%time
get_closest_match_with_metric('сонце', WORDS, 5, textdistance.damerau_levenshtein)

CPU times: user 14.7 s, sys: 0 ns, total: 14.7 s
Wall time: 14.8 s


[('солнце', 0.8333333333333334),
 ('конце', 0.8),
 ('соне', 0.8),
 ('синее', 0.8),
 ('сложнее', 0.7142857142857143)]

С большим словарем даже оптимизированные версии будут работать очень долго. Давайте попробуем избавиться от необходимости сравнивать слово со всеми в словаре.

Многие вещи, которые медленно решаются в питоне, можно оптимизировать с помощью матричных и векторных операций.

Сделаем поиск похожих по векторам символов, из которых состоит слово. Косинусное расстояние между векторами слов не равно расстоянию редактирования, т.к. в нем не учитывается порядок символов. Близкими будут слова, состоящие из одинаковых символов. Но мы можем быть уверены, что близкими точно не будут слова с большим косинсным расстоянием, и можем их не проверять их честной метрикой редактирования.

In [185]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [186]:

WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

Мы ещё подробно разберем векторизацию, пока просто позапускайте код ниже, чтобы понять, какие параметры на что вляют и что получается в итоге.

In [193]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

# в sklearn 2 основных векторизатора - CountVectorizer, TfidfVectorizer
# первый превращает текст в вектора по частотности слов (текст "один один два" - вектор [2, 1])
# во втором частотности умножаются на idf слова

# в векторайзер тексты подаются списком строк
# он сам сделает предобработку и токенизацию

#т.к. у нас вместо текстов отдельные слова, мы ставим analyzer='char'
# т.е. слова рассматриваются как последовательность символов
# в ngram_range=(1,n) можно задать размер символьных нграммов, которые хотим получить
# мы ставим 1,1 т.к нам нужны отдельные символы
#min_df - это минимальное количество документов (слов в нашем случае), в которых должен встретиться символ, 
# чтобы попасть в словарь
vec = CountVectorizer(analyzer='char', ngram_range=(1,1), min_df=10)
X = vec.fit_transform(vocab)

In [196]:
def get_closest_match_vec(text, X, vec, topn=20):
    # превращаем слово в вектор такой же размерности
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0] #distance - чем больше, тем хуже, а similarity наоборот
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [198]:
%%time
get_closest_match_vec('сонце', X, vec) # это расстояние - чем меньше тем лучше

CPU times: user 52 ms, sys: 4 ms, total: 56 ms
Wall time: 55.1 ms


[('сценой', 0.0871290708247231),
 ('сенцов', 0.0871290708247231),
 ('солнце', 0.0871290708247231),
 ('ценностное', 0.10000000000000009),
 ('самоценное', 0.10557280900008414),
 ('бессоннице', 0.10557280900008414),
 ('сцен', 0.10557280900008414),
 ('соне', 0.10557280900008414),
 ('броненосец', 0.10557280900008414),
 ('неонацистское', 0.12168993434632014),
 ('концессионеров', 0.12294198069297058),
 ('миноносец', 0.1317568578755408),
 ('расценено', 0.1317568578755408),
 ('оценено', 0.1317568578755408),
 ('соццентре', 0.1317568578755408),
 ('ценностей', 0.1317568578755408),
 ('бессонице', 0.1317568578755408),
 ('крестоносец', 0.13227816872537534),
 ('броненосцем', 0.13227816872537534),
 ('бесцеремонно', 0.14188366967896693)]

Напишем функцию, которая принимает слово и находит ближайшее к нему в словаре с помощью косисного расстояния и честной метрики редактирования. 

In [204]:
def get_closest_hybrid_match(text, X, vec, topn=5, metric=textdistance.damerau_levenshtein):
    # ваш код здесь
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    sims = Counter()
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup,topn, metric=metric)

    
    return closest

In [205]:
%time
get_closest_hybrid_match('сонце', X, vec)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 16.2 µs


[('солнце', 0.8333333333333334),
 ('соне', 0.8),
 ('соццентре', 0.5555555555555556),
 ('бессонице', 0.5555555555555556),
 ('сенцов', 0.5)]

Оценим такой метод исправления.

In [208]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        pred = get_closest_hybrid_match(pair[1], X, vec)[0][0]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

In [209]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8454545454545455
0.4712202609363008
0.09854140346847365


Процент неправильно исправленных снизился и теперь общий процент правильных почти равен проценту изначально правильных.

Чтобы улучшить качество исправлений, мы можем попробовать учитывать контекст. В этом нам помогут языковые модели. Мы можем обучить языковую модель на нашем новостном корпусе и потом домножать каждое расстояние на вероятность.

In [123]:

corpus_news = [['<start>', '<start>'] + sent + ['<end>'] for sent in corpus]

In [124]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [125]:
unigrams = Counter()
bigrams = Counter()

for sentence in corpus_news:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))


In [126]:
# оцените качество также как и раньше
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0



for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    word_pairs = [('<start>', '<start>')] + word_pairs
    pred_sent = []
    for j in range(1, len(word_pairs)):
        
        pred = None
        predicted = get_closest_hybrid_match(word_pairs[j][1], X, vec)
        
        
        prev_word = word_pairs[j-1][1]
        
        
        if prev_word not in unigrams:
            pred = predicted[0][0]
            
        
        else:
            
            lm_predicted = []
            for word, m in predicted:
                bigram = ' '.join([prev_word, word])
                # домножаем полученную метрику для слова на вероятность биграма
                # биграм - предыдущее слово + текущее слово кандидат
                lm_predicted.append((word, (m)*(1+(bigrams[bigram]/unigrams[prev_word]))))
            if lm_predicted:
                
                pred = sorted(lm_predicted, key=lambda x: -x[1])[0][0]
            
        
        if pred is None:
            pred = word_pairs[j][1]
        

        
        if pred == word_pairs[j][0]:
            correct += 1
        else:
            mistakes.append((word_pairs[j][0], word_pairs[j][1], pred))
        total += 1
            
        if word_pairs[j][0] == word_pairs[j][1]:
            total_correct += 1
            if word_pairs[j][0] !=  pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if word_pairs[j][0] == pred:
                mistaken_fixed += 1
    
    if not i % 50:
        print(i)
        print(correct/total)

0
0.8666666666666667
50
0.8389694041867954
100
0.8324372759856631
150
0.844106463878327
200
0.8465608465608465
250
0.8512669849430775
300
0.8482572115384616
350
0.8458518712378958
400
0.848295059151009
450
0.8479166666666667
500
0.8467682020802377
550
0.8479293957909029
600
0.8498072474942174
650
0.8513226764747489
700
0.850262467191601
750
0.8480717268484402
800
0.847005884389062
850
0.8466343564889607
900
0.8482632541133455


In [127]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8487512487512487
0.49884881043745205
0.09888595383025152


К сожалению процент сломанных не снизился, а процент правильно исправленных повысился незначительно. Возможно на нужен другой корпус или другой алгоритм определения ошибок.

## Домашнее задание

1. Реализуйте алгоритм [Symspell](https://medium.com/@wolfgarbe/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f). Он похож на алгоритм Норвига, но проще и быстрее. Там к словам в словаре применяется только одна операция - удаление символа (1-n). Чтобы найти исправление из слова тоже удаляются символы и сравниваются с теми, что хранятся в словаре. Оцените качество полученного алгоритма теми же тремя метриками.

2. Добавьте к полученному алгоритму исправления (Symspell) триграммную модель и проверьте, улучшает ли она качество. Триграммную модель нужно вставить туда, где у вас выбирается один из нескольких кандидатов на исправление.

Оформите все в одну (!) jupyter-тетрадку, загрузите на гитхаб, nbviewer или google.drive и сдайте через эту форму - https://forms.gle/FLHrnM3yKJiNw6bN8

Если у вас возникаются какие-то проблемы с выполнением задания, сразу пишите мне в телеграм (@Nefedov_m). 